In [1]:
import importlib.util
import os
from pyspark.sql.functions import broadcast


# Load preprocessing module
spec = importlib.util.spec_from_file_location("utils", '../scripts/preprocessing.py')
preprocess = importlib.util.module_from_spec(spec)
spec.loader.exec_module(preprocess)
print(preprocess.test())
#imports
import os
import pwd
import numpy as np
import sys
import pandas as pd

from pyspark.sql import SparkSession
from random import randrange
import pyspark.sql.functions as F
#np.bool = np.bool_
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import (StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler)
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.types import DoubleType, IntegerType, DateType
from pyspark.sql.functions import rank, col, avg, date_format, count, year, expr, coalesce, lit, to_timestamp, unix_timestamp, hour,when
from sklearn.ensemble        import GradientBoostingRegressor
from sklearn.metrics         import mean_squared_error, r2_score
import base64 as b64
import json
import time
import re

import os
import warnings


import trino
from contextlib import closing
from urllib.parse import urlparse
from trino.dbapi import connect
from trino.auth import BasicAuthentication, JWTAuthentication
#setup spark session and trino
username = pwd.getpwuid(os.getuid()).pw_name
hadoopFS=os.getenv('HADOOP_FS', None)
groupName = 'U1'
print(os.getenv('SPARK_HOME'))
print(f"hadoopFSs={hadoopFS}")
print(f"username={username}")
print(f"group={groupName}")

spark = SparkSession\
            .builder\
            .appName(pwd.getpwuid(os.getuid()).pw_name)\
            .config('spark.ui.port', randrange(4040, 4440, 5))\
            .config("spark.executorEnv.PYTHONPATH", ":".join(sys.path)) \
            .config('spark.jars', f'{hadoopFS}/data/com-490/jars/iceberg-spark-runtime-3.5_2.13-1.6.1.jar')\
            .config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')\
            .config('spark.sql.catalog.iceberg', 'org.apache.iceberg.spark.SparkCatalog')\
            .config('spark.sql.catalog.iceberg.type', 'hadoop')\
            .config('spark.sql.catalog.iceberg.warehouse', f'{hadoopFS}/data/com-490/iceberg/')\
            .config('spark.sql.catalog.spark_catalog', 'org.apache.iceberg.spark.SparkSessionCatalog')\
            .config('spark.sql.catalog.spark_catalog.type', 'hadoop')\
            .config('spark.sql.catalog.spark_catalog.warehouse', f'{hadoopFS}/user/{username}/assignment-3/warehouse')\
            .config("spark.sql.warehouse.dir", f'{hadoopFS}/user/{username}/assignment-3/spark/warehouse')\
            .config('spark.eventLog.gcMetrics.youngGenerationGarbageCollectors', 'G1 Young Generation')\
            .config("spark.executor.memory", "6g")\
            .config("spark.executor.cores", "4")\
            .config("spark.executor.instances", "4")\
            .master('yarn')\
            .getOrCreate()

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.filterwarnings("ignore", category=UserWarning, message="pandas only supports SQLAlchemy connectable .*")



def getUsername():
    payload = os.environ.get('EPFL_COM490_TOKEN').split('.')[1]
    payload=payload+'=' * (4 - len(payload) % 4)
    obj = json.loads(b64.urlsafe_b64decode(payload))
    if (time.time() > int(obj.get('exp')) - 3600):
        raise Exception('Your credentials have expired, please restart your Jupyter Hub server:'
                        'File>Hub Control Panel, Stop My Server, Start My Server.')
    time_left = int((obj.get('exp') - time.time())/3600)
    return obj.get('sub'), time_left

username, validity_h = getUsername()
hadoopFS = os.environ.get('HADOOP_FS')
namespace = 'iceberg.' + username
sharedNS = 'iceberg.com490_iceberg'

if not re.search('[A-Z][0-9]', groupName):
    raise Exception('Invalid group name {groupName}')

print(f"you are: {username}")
print(f"credentials validity: {validity_h} hours left.")
print(f"shared namespace is: {sharedNS}")
print(f"your namespace is: {namespace}")
print(f"your group is: {groupName}")

trinoAuth = JWTAuthentication(os.environ.get('EPFL_COM490_TOKEN'))
trinoUrl  = urlparse(os.environ.get('TRINO_URL'))
Query=[]

print(f"Warehouse URL: {trinoUrl.scheme}://{trinoUrl.hostname}:{trinoUrl.port}/")

conn = connect(
    host=trinoUrl.hostname,
    port=trinoUrl.port,
    auth=trinoAuth,
    http_scheme=trinoUrl.scheme,
    verify=True
)

print('Connected!')

Imported successfully!
/opt/spark
hadoopFSs=hdfs://iccluster059.iccluster.epfl.ch:9000
username=omanovic
group=U1


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/24 17:56:56 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


you are: omanovic
credentials validity: 167 hours left.
shared namespace is: iceberg.com490_iceberg
your namespace is: iceberg.omanovic
your group is: U1
Warehouse URL: https://iccluster028.iccluster.epfl.ch:8443/
Connected!


In [7]:
raw = spark.read.parquet(f"{hadoopFS}/user/com-490/group/U1/raw_df_modelling.parquet")

In [23]:
raw.printSchema()
raw.show(1)

root
 |-- stop_id: string (nullable = true)
 |-- operating_day: string (nullable = true)
 |-- dow: string (nullable = true)
 |-- trip_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_actual: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_actual: string (nullable = true)
 |-- arr_delay_sec: string (nullable = true)
 |-- dep_delay_sec: string (nullable = true)
 |-- site: string (nullable = true)
 |-- total_daily_precip: string (nullable = true)
 |-- rained: string (nullable = true)
 |-- stop_name: string (nullable = true)
 |-- stop_lat: string (nullable = true)
 |-- stop_lon: string (nullable = true)

+-------+-------------+---+-----------+-----+--------------------+--------------------+--------------------+--------------------+-------------+-------------+----+------------------+------+---------------+--------------------+--------------------+
|stop_id|operating_day|dow|    trip_id| type|         

In [33]:
from pyspark.sql.types    import DoubleType, IntegerType, DateType, TimestampType
from pyspark.sql.functions import (
    to_timestamp, unix_timestamp, col,
    hour, when, date_format
)
from pyspark.ml            import Pipeline
from pyspark.ml.feature    import (
    StringIndexer, OneHotEncoder,
    VectorAssembler
)
from pyspark.ml.regression import GBTRegressor

# 2) Parse timestamps & recompute arr_delay_sec
df = raw \
  .withColumn("arr_time_ts",   to_timestamp("arr_time")) \
  .withColumn("arr_actual_ts", to_timestamp("arr_actual")) \
  .withColumn("dep_time_ts",   to_timestamp("dep_time")) \
  .withColumn("dep_actual_ts", to_timestamp("dep_actual")) \
  .withColumn("arr_delay_sec",
      ( unix_timestamp("arr_actual_ts") - unix_timestamp("arr_time_ts") )
        .cast(DoubleType())
   ) \
  .filter(col("arr_delay_sec").between(-300, 600))

# 3) Cast numeric strings to doubles/ints
features = df \
  .withColumn("total_daily_precip", col("total_daily_precip").cast(DoubleType())) \
  .withColumn("stop_lat",           col("stop_lat").cast(DoubleType())) \
  .withColumn("stop_lon",           col("stop_lon").cast(DoubleType())) \
  .withColumn("dep_hour",           hour("dep_time_ts").cast(IntegerType()))

# 4) Build simple engineered features
features = features \
  .withColumn("is_raining", when(col("total_daily_precip") > 0, 1).otherwise(0)) \
  .withColumn("dow_str",    date_format(col("operating_day").cast(DateType()), "E"))

# 5) Select the columns you need and sample a small subset for debugging (1%)
sampled = features.select(
    "trip_id","stop_id","arr_delay_sec",
    "dep_hour","total_daily_precip","is_raining",
    "stop_lat","stop_lon",
    "dow_str","type","stop_name"
).sample(False, 0.01, seed=42)

# 6) Index & one‐hot encode all categoricals
dow_idx      = StringIndexer(inputCol="dow_str",    outputCol="dow_idx",      handleInvalid="keep")
dow_ohe      = OneHotEncoder(inputCol="dow_idx",     outputCol="dow_vec")
type_idx     = StringIndexer(inputCol="type",       outputCol="type_idx",     handleInvalid="keep")
type_ohe     = OneHotEncoder(inputCol="type_idx",    outputCol="type_vec")
stopname_idx = StringIndexer(inputCol="stop_name",  outputCol="stopname_idx", handleInvalid="keep")
stopname_ohe = OneHotEncoder(inputCol="stopname_idx",outputCol="stopname_vec")

# 7) Assemble everything into “features”
assembler = VectorAssembler(
    inputCols=[
      "dep_hour","total_daily_precip","is_raining",
      "stop_lat","stop_lon",
      "dow_vec","type_vec","stopname_vec"
    ],
    outputCol="features"
)

# 8) Pick a lighter regressor for debug (e.g. small GBT)
gbt = GBTRegressor(
    featuresCol="features",
    labelCol="arr_delay_sec",
    maxDepth=5,
    maxIter=20,
    stepSize=0.2
)

# 9) Build the pipeline
pipeline = Pipeline(stages=[
    dow_idx, dow_ohe,
    type_idx, type_ohe,
    stopname_idx, stopname_ohe,
    assembler,
    gbt
])

# 10) Train/test split on the sampled data
train, test = sampled.randomSplit([0.8,0.2], seed=42)

# 11) Fit & predict (should finish in seconds)
model = pipeline.fit(train)
pred  = model.transform(test)

# 12) Inspect a few results
pred.select("trip_id","stop_id","stop_name","arr_delay_sec","prediction") \
    .show(10, truncate=False)


+---------------+-------+--------------------------+-------------+------------------+
|trip_id        |stop_id|stop_name                 |arr_delay_sec|prediction        |
+---------------+-------+--------------------------+-------------+------------------+
|85:11:18416:002|8501120|Lausanne                  |17.0         |12.403536300524877|
|85:11:24462:001|8501120|Lausanne                  |15.0         |38.68613081535594 |
|85:11:2511:003 |8501120|Lausanne                  |-56.0        |38.5291510476241  |
|85:151:1001    |8592053|Lausanne, Grande-Borde    |-75.0        |88.33627450208508 |
|85:151:1006    |8592102|Lausanne, Prélaz          |36.0         |58.94811914538086 |
|85:151:101     |8592005|Lausanne, Cèdres          |-41.0        |52.21547490135334 |
|85:151:1020    |8592001|Lausanne, Boston          |-71.0        |84.76855877864236 |
|85:151:1035    |8592103|Lausanne, Prélaz-les-Roses|232.0        |75.19760128231327 |
|85:151:1052    |8592001|Lausanne, Boston          |19

In [34]:
# 8) EVALUATE
evaluator = RegressionEvaluator(
    labelCol="arr_delay_sec",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(pred, {evaluator.metricName: "rmse"})
mae  = evaluator.evaluate(pred, {evaluator.metricName: "mae"})
r2   = evaluator.evaluate(pred, {evaluator.metricName: "r2"})

print(f"RMSE = {rmse:.2f} s")
print(f"MAE  = {mae:.2f} s")
print(f"R²   = {r2:.4f}")

RMSE = 129.67 s
MAE  = 95.36 s
R²   = 0.0518


In [35]:
pred.write.parquet(f"{hadoopFS}/user/com-490/group/U1/predictions_regression.parquet", mode="overwrite")

In [42]:
pred.printSchema()

root
 |-- trip_id: string (nullable = true)
 |-- stop_id: string (nullable = true)
 |-- arr_delay_sec: double (nullable = true)
 |-- dep_hour: integer (nullable = true)
 |-- total_daily_precip: double (nullable = true)
 |-- is_raining: integer (nullable = false)
 |-- stop_lat: double (nullable = true)
 |-- stop_lon: double (nullable = true)
 |-- dow_str: string (nullable = true)
 |-- type: string (nullable = true)
 |-- stop_name: string (nullable = true)
 |-- dow_idx: double (nullable = false)
 |-- dow_vec: vector (nullable = true)
 |-- type_idx: double (nullable = false)
 |-- type_vec: vector (nullable = true)
 |-- stopname_idx: double (nullable = false)
 |-- stopname_vec: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [ ]:
# ── 1) Re‐load just the raw df_final, no added cols ────────────────────────
raw = spark.read.parquet(f"{hadoopFS}/user/com-490/group/U1/raw_df_modelling.parquet").select(
    "stop_id","operating_day","dow","trip_id","type",
    "arr_time","arr_actual","dep_time","dep_actual",
    "arr_delay_sec","total_daily_precip","rained",
    "stop_name","stop_lat","stop_lon"
)

# 2) Cast & basic feature‐engineering
df = (
    raw
    .withColumn("arr_delay_sec",      col("arr_delay_sec"). cast(DoubleType()))
    .withColumn("total_daily_precip", col("total_daily_precip").cast(DoubleType()))
    .withColumn("stop_lat",           col("stop_lat").          cast(DoubleType()))
    .withColumn("stop_lon",           col("stop_lon").          cast(DoubleType()))
    .withColumn("arr_time_ts",        to_timestamp("arr_time"))
    .na.drop(subset=[
        "arr_delay_sec","total_daily_precip",
        "stop_lat","stop_lon","arr_time_ts"
    ])
    # only Mon–Fri, hours 7–20
    .withColumn("dow", col("dow").cast(IntegerType()))
    .filter(col("dow").between(1,5))
    .withColumn("hour", hour("arr_time_ts"))
    .filter(col("hour").between(7,20))
    .withColumn("is_raining", when(col("total_daily_precip")>0,1).otherwise(0))
    .withColumn("dow_str",
        date_format(col("operating_day").cast(DateType()), "E")
    )
    .na.drop()
)

# 3) Build the shared “features only” pipeline
#    (same as before: index dow/type/stop_name → OHE → assemble + cache)
dow_idx      = StringIndexer(inputCol="dow_str",   outputCol="dow_idx",  handleInvalid="keep")
dow_ohe      = OneHotEncoder(inputCol="dow_idx",    outputCol="dow_vec")
type_idx     = StringIndexer(inputCol="type",      outputCol="type_idx", handleInvalid="keep")
type_ohe     = OneHotEncoder(inputCol="type_idx",   outputCol="type_vec")
stop_idx     = StringIndexer(inputCol="stop_name", outputCol="stop_idx", handleInvalid="keep")
stop_ohe     = OneHotEncoder(inputCol="stop_idx",   outputCol="stop_vec")

assembler = VectorAssembler(
    inputCols=[
      "dow_vec","type_vec","stop_vec",
      "hour","total_daily_precip","is_raining",
      "stop_lat","stop_lon"
    ],
    outputCol="features"
)

feat_pipe  = Pipeline(stages=[
    dow_idx, dow_ohe,
    type_idx, type_ohe,
    stop_idx, stop_ohe,
    assembler
])

feat_model = feat_pipe.fit(df)
df_feats   = feat_model.transform(df) \
             .select("trip_id","stop_id","features","arr_delay_sec") \
             .cache()
df_feats.count()

# 4)  Train a **sign** classifier: `label = 1 if arr_delay_sec≥0 else 0`
sign_df = df_feats.withColumn(
    "is_late", (col("arr_delay_sec") >= 0).cast(IntegerType())
)

sign_clf = GBTClassifier(
    labelCol="is_late", featuresCol="features",
    maxIter=20, maxDepth=5, stepSize=0.1, seed=42
)

sign_model = sign_clf.fit(sign_df)

# 5)  Train a single **magnitude** regressor on absolute delay
mag_df = df_feats.withColumn("abs_delay", sql_abs(col("arr_delay_sec")))

mag_reg = GBTRegressor(
    labelCol="abs_delay", featuresCol="features",
    maxIter=50, maxDepth=5, stepSize=0.1, seed=42
)

mag_model = mag_reg.fit(mag_df)

# 6)  At inference, score both **once**:
scored = sign_model.transform(df_feats) \
          .withColumnRenamed("probability","sign_prob") \
          .withColumnRenamed("prediction","sign_pred")

scored = mag_model.transform(scored) \
          .withColumnRenamed("prediction","mag_pred")

# 7)  Combine into final signed prediction
final = scored.withColumn(
    "final_pred",
    when(col("sign_prob")[1] >= 0.5, col("mag_pred"))
   .otherwise(-col("mag_pred"))
)

# 8)  Inspect
final.select(
    "trip_id","stop_id",
    "arr_delay_sec",
    "sign_prob","mag_pred","final_pred"
).show(10,False)



In [ ]:
pred_late.dropDuplicates().show(10)
pred_early.dropDuplicates().show(10)

In [32]:
small_spark = (
    raw
    .withColumn("arr_delay_sec",       F.col("arr_delay_sec").cast("double"))
    .withColumn("total_daily_precip",  F.col("total_daily_precip").cast("double"))
    .withColumn("hour",                F.hour(F.to_timestamp("arr_time")).cast("int"))
    .withColumn("dow_str",             F.date_format(F.to_timestamp("operating_day"), "E"))
    .withColumn("is_raining",          F.when(F.col("total_daily_precip") > 0, 1).otherwise(0))
    # restrict to Mon–Fri, hours 07–20
    .filter(F.col("dow_str").isin("Mon","Tue","Wed","Thu","Fri"))
    .filter(F.col("hour").between(7,20))
    .select(
        "arr_delay_sec",
        "hour",
        "total_daily_precip",
        "is_raining",
        "dow_str",
        "type",
        "stop_name",
        "stop_lat",
        "stop_lon"
    )
    .sample(False, 0.02, seed=42)   # just 2% sample for speed
    .limit(50000)                   # cap at 50 k rows
)

# --- 2) move into pandas for clustering & interaction features ---
pdf = small_spark.toPandas()

# interaction terms
pdf["hour_precip"] = pdf["hour"] * pdf["total_daily_precip"]
pdf["hour_rain"]  = pdf["hour"] * pdf["is_raining"]

# cluster stops into 10 “area” buckets
from sklearn.cluster import KMeans
coords = pdf[["stop_lat","stop_lon"]].astype(float)
km = KMeans(n_clusters=10, random_state=42).fit(coords)
pdf["area"] = km.labels_.astype(str)

# --- 3) split into train/test ---
from sklearn.model_selection import train_test_split
y = pdf["arr_delay_sec"].values
X = pdf.drop(columns="arr_delay_sec")
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 4) define columns for preprocessing ---
cat_cols = ["dow_str","type","stop_name","area"]
num_cols = ["hour","total_daily_precip","is_raining",
            "hour_precip","hour_rain"]

# --- 5) build the sklearn pipeline ---
from sklearn.pipeline       import Pipeline
from sklearn.compose        import ColumnTransformer
from sklearn.preprocessing  import OneHotEncoder, StandardScaler
from sklearn.ensemble       import GradientBoostingRegressor

preprocessor = ColumnTransformer([
    ("cats", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ("nums", StandardScaler(), num_cols),
])

# --- 6) train one quantile‐regressor per target quantile ---
quantiles = [0.1, 0.5, 0.9]
models = {}
for q in quantiles:
    gbr = GradientBoostingRegressor(
        loss="quantile",
        alpha=q,
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        random_state=42
    )
    pipe = Pipeline([
        ("pre", preprocessor),
        ("gbr", gbr)
    ])
    pipe.fit(Xtr, ytr)
    models[q] = pipe

# --- 7) assemble out‐of‐sample quantile predictions ---
import pandas as pd
preds = pd.DataFrame({"actual": yte})
for q, pipe in models.items():
    preds[f"q{int(q*100)}"] = pipe.predict(Xte)

# quick coverage check
for q in quantiles:
    col = f"q{int(q*100)}"
    if q < 0.5:
        coverage = (preds["actual"] >= preds[col]).mean()
    else:
        coverage = (preds["actual"] <= preds[col]).mean()
    print(f"α={q:.2f}: empirical coverage ≈ {coverage:.2f}")

# peek
print(preds.head())

α=0.10: empirical coverage ≈ 0.90
α=0.50: empirical coverage ≈ 0.50
α=0.90: empirical coverage ≈ 0.90
   actual        q10        q50         q90
0  -676.0 -41.051983  51.063325  138.816753
1   153.0 -50.635623  63.343375  309.051633
2   -12.0 -16.145387   3.421079   57.356084
3   110.0 -54.301806  54.036515  252.750592
4    75.0 -65.721357  58.107326  280.471712


In [15]:
from pyspark.sql.functions import (
    to_timestamp, unix_timestamp, from_unixtime,
    col, lead
)
from pyspark.sql.window import Window
import heapq
from datetime import datetime

preds2 = (
    pred
    .filter(col("operating_day") == "2024-01-10")    
    .withColumn("t_dep",   to_timestamp("dep_time"))  \
    .withColumn("delay_s", col("prediction"))        \
    .filter(col("t_dep").isNotNull() & col("delay_s").isNotNull())
)

w = Window.partitionBy("trip_id").orderBy("t_dep")
segments = (
    preds2
    .withColumn("next_stop", lead("stop_id").over(w))
    .withColumn("t_eff",
        from_unixtime(
            unix_timestamp("t_dep") + col("delay_s")
        ).cast("timestamp")
    )
    .filter(col("next_stop").isNotNull())
    .select("stop_id","next_stop","t_dep","t_eff","prediction") 
)

edges = segments.collect()
graph_inc = {}
for eid, r in enumerate(edges):
    u       = r["stop_id"]
    v       = r["next_stop"]
    t_dep   = r["t_dep"]    
    t_eff   = r["t_eff"]    
    p_on    = 1.0           
    graph_inc.setdefault(v, []).append((u, t_dep, t_eff, p_on, eid))


def k_latest_routes(graph_inc, source, target, T, K=5):
    """
    Returns up to K tuples of (dep_time, path_edges, confidence)
      - dep_time: datetime of departure at `source`
      - path_edges: list of (u, v, edge_index) from source→…→target
      - confidence: product of per-edge p_on
    """
    # max-heap: store (–dep_ts, node, path_so_far, conf_so_far)
    heap = [(-T.timestamp(), target, [], 1.0)]
    found = []

    while heap and len(found) < K:
        neg_ts, v, path, conf = heapq.heappop(heap)
        t_v = datetime.fromtimestamp(-neg_ts)

        if v == source:
            # we have a full path from source→...→target
            found.append((t_v, list(reversed(path)), conf))
            continue

        for u, t_dep, t_eff, p_on, eid in graph_inc.get(v, []):
            # can only catch that segment if it gets you in time
            if t_eff <= t_v:
                new_conf = conf * p_on
                new_path = path + [(u, v, eid)]
                heapq.heappush(heap, (-t_dep.timestamp(), u, new_path, new_conf))

    return found

A = "8501120"
B = "8501121"
T = datetime(2024, 1, 10, 18, 10)

routes = k_latest_routes(graph_inc, A, B, T, K=3)
if not routes:
    print(f"No feasible route from {A} → {B} by {T}")
else:
    for dep_time, path, conf in routes:
        print(f"\nDepart {A} @ {dep_time.isoformat()}  (confidence {conf:.2%})")
        for u, v, eid in path:
            seg = edges[eid]
            print(f"   {u} → {v}   dep@{seg['t_dep']}   arr_eff@{seg['t_eff']}")


No feasible route from 8501120 → 8501121 by 2024-01-10 18:10:00
